## KB 사용자 모의 데이터 생성

### Library

In [2]:
import random
import pandas as pd
import numpy as np
import os
from pprint import pprint
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

### config

In [4]:
input_path = os.path.join(os.getcwd(), "base_data") 
save_path = os.path.join(os.getcwd(), "data")
os.makedirs(save_path, exist_ok=True)

### Data Load

In [ ]:
# 파일 호출 - 단위는 모두 (천명) - 연도는 모두 2022년 기준
address_nojob = pd.read_csv(os.path.join(input_path, "address_nojob.csv"), encoding="euc-kr", header=0) 
gender_nojob = pd.read_csv(os.path.join(input_path, "gender_nojob.csv"), encoding="euc-kr", header=0) 
gender_job = pd.read_csv(os.path.join(input_path, "gender_job.csv"), encoding="euc-kr", header=0) 
address_nojob.shape, gender_nojob.shape, gender_job.shape

### 비율산출

### 1. 성별/직업군별/행정구역별 - 경제활동인구
- gender_job

In [ ]:
# 수도권 생성 - 서울특별시: 서울 전체 직업군 합계, .1:관리자, .2:전문가 및 관련종사자, .3: 사무 종사자, .4: 서비스 종사자, .5:판매 종사자,
#               .6: 농림어업 숙련 종사자, .7: 기능원 및 관련 기능종사자, .8: 장치기계 조작 및 조립 종사자, .9: 단순 노무 종사자, .10: 기타직업

gj = [gender_job.iloc[:, i:i+1].columns.to_list() for i in range(3, 190, 11)] # 지역명 추출
gj = sum(gj, []) # 2차원 리스트 -> 1차원 리스트

# 전국의 경제활동인구
total = sum(gender_job.loc[1, gj].astype("int"))

# 17개의 행정구역으로 나누어 주는 함수
def gj_list():    
    m_region_zip = [gender_job.loc[2, region:region+".10"].astype("int").to_list() for region in gj]
    f_region_zip = [gender_job.loc[3, region:region+".10"].astype("int").to_list() for region in gj]
    
    m_region_rate_zip = [[(m_region[i]/total) for i in range(0, len(m_region))] for m_region in m_region_zip]
    f_region_rate_zip = [[(f_region[i]/total) for i in range(0, len(f_region))] for f_region in f_region_zip]
    
    region_rate_zip = [m_region_rate_zip, f_region_rate_zip]
    
    return region_rate_zip
region_rate_zip = gj_list()

# 수도권, 강원도, 충청도, 경상도, 전라도, 제주도 -> 권역별 합치기
capital_area_rate = [[(mf[0][i]+mf[3][i]+mf[8][i]) for i in range(0, len(mf[0]))] for mf in region_rate_zip]  # 수도권
gangwondo_rate = [[(mf[9][i]) for i in range(0, len(mf[9]))] for mf in region_rate_zip]  # 강원도
chungcheongdo_rate = [[(mf[5][i]+mf[7][i]+mf[10][i]+ \
    mf[11][i]) for i in range(0, len(mf[5]))] for mf in region_rate_zip]  # 충청도
gyeongsangdo_rate = [[(mf[1][i]+mf[2][i]+mf[6][i]+mf[14][i]+ \
    mf[15][i]) for i in range(0, len(mf[1]))] for mf in region_rate_zip] # 경상도
jeollado_rate = [[(mf[4][i]+mf[12][i]+mf[13][i]) for i in range(0, len(mf[4]))] for mf in region_rate_zip]  # 전라도
jejudo_rate = [[(mf[16][i]) for i in range(0, len(mf[16]))] for mf in region_rate_zip]  # 제주도

# 데이터프레임 생성 - 경제활동인구(수도권, 강원도, 충청도, 경상도, 제주도) --> 비경제활동인구(수도권, 강원도, 충청도, 경상도, 제주도)
region_list = [capital_area_rate[0], gangwondo_rate[0], chungcheongdo_rate[0], gyeongsangdo_rate[0], jeollado_rate[0], jejudo_rate[0],
               capital_area_rate[1], gangwondo_rate[1], chungcheongdo_rate[1], gyeongsangdo_rate[1], jeollado_rate[1], jejudo_rate[1]]
gj_df = pd.DataFrame()
for i in region_list:
    sub_df = pd.DataFrame([i], columns=["total", "관리자", "전문가_및_관련종사자", "사무_종사자", "서비스_종사자", "판매_종사자", 
                                        "농림어업_숙련_종사자", "기능원_및_관련기능_종사자", "장치_기계조작_및_조립_종사자", "단순_노무_종사자", "기타직업"])
    gj_df = pd.concat([gj_df, sub_df], axis=0)
    gj_df.reset_index(drop=True, inplace=True)

# 0~5는 남자, 6~11은 여자

### 2. 거주지별 비경제활동인구
- address_nojob

In [ ]:
# 17개의 행정구역별 전체대비 비율
def ad_nj():
    total = address_nojob.iloc[1, 1:8].astype("int").to_list()
    total = [total[0], total[1], total[2], total[3], sum(total[4:])]  # 전국 총합 인원
    total_rate = [(total[i]/total[0]) for i in range(0, len(total))]
    
    region_zip = [address_nojob.iloc[1, i:i+5].astype("int").to_list() for i in range(8, 89, 5)]
    region_rate_zip = [[(region[i]/total[0]) for i in range(0, len(region))] for region in region_zip]
    return total_rate, region_rate_zip, region_zip

total_rate, region_rate_zip, region_zip = ad_nj()

# 수도권, 강원도, 충청도, 경상도, 전라도, 제주도 -> 권역별 합치기
capital_area_rate = [(region_rate_zip[0][i]+region_rate_zip[3][i]+region_rate_zip[8][i]) for i in range(0, len(region_rate_zip[0]))]  # 수도권
gangwondo_rate = [(region_rate_zip[9][i]) for i in range(0, len(region_rate_zip[9]))]  # 강원도
chungcheongdo_rate = [(region_rate_zip[5][i]+region_rate_zip[7][i]+region_rate_zip[10][i]+ \
    region_rate_zip[11][i]) for i in range(0, len(region_rate_zip[5]))]  # 충청도
gyeongsangdo_rate = [(region_rate_zip[1][i]+region_rate_zip[2][i]+region_rate_zip[6][i]+region_rate_zip[14][i]+ \
    region_rate_zip[15][i]) for i in range(0, len(region_rate_zip[1]))] # 경상도
jeollado_rate = [(region_rate_zip[4][i]+region_rate_zip[12][i]+region_rate_zip[13][i]) for i in range(0, len(region_rate_zip[4]))]  # 전라도
jejudo_rate = [(region_rate_zip[16][i]) for i in range(0, len(region_rate_zip[16]))]  # 제주도

# 권역별 비경제활동 사유별 전체 비중 비율 데이터프레임으로 만들기
rate_list = [total_rate, capital_area_rate, gangwondo_rate, chungcheongdo_rate, gyeongsangdo_rate, jeollado_rate, jejudo_rate]  # 권역별 비율 모음
rate_no_job = pd.DataFrame(columns=["total", "house1", "house2", "student", "etc"])
# house 1은 가정, house 2는 육아 -> 추후 house로 합쳐 주부 비율로 활용

for rate in rate_list:
    sub_df = pd.DataFrame([rate], columns=["total", "house1", "house2", "student", "etc"])
    rate_no_job = pd.concat([rate_no_job, sub_df], axis=0)
rate_no_job.reset_index(drop=True, inplace=True)
rate_no_job["house"] = rate_no_job["house1"]+rate_no_job["house2"]  # 주부 비율로 합치기
rate_no_job.drop(["house1", "house2"], axis=1, inplace=True)

### 3. 성별/활동상태별 비율
- gender_nojob

In [ ]:
# 리스트 생성 [2]
total = gender_nojob.iloc[1, 1:9].astype("int").to_list() # 성별 구분 없는 항목별 총합
male = gender_nojob.iloc[1, 9:17].astype("int").to_list() # 남성만 분리
female = gender_nojob.iloc[1, 17:26].astype("int").to_list() # 여성만 분리

total_rate = [total[i]/total[i] for i in range(0, len(total))] # 비율을 위한 계산
male_rate = [male[i]/total[i] for i in range(0, len(male))] # 항목별 전체인구 대비 남성인구의 비율
female_rate = [female[i]/total[i] for i in range(0, len(female))] # 항목별 전체인구 대비 여성인구 비율

no_list = [total_rate, male_rate, female_rate] # 전체, 남성, 여성 순으로 나열
no_job = pd.DataFrame(columns=["total", "house1", "house2", "student", "etc1", "etc2", "etc3", "etc4"]) # 비경제활동 요인별 열이름 구성
for no in no_list:
    sub_df = pd.DataFrame([no], columns=["total", "house1", "house2", "student", "etc1", "etc2", "etc3", "etc4"]) 
    # 전체, 남성, 여성 순으로 들어올 리스트들의 데이터프레임 구성
    no_job = pd.concat([no_job, sub_df], axis=0) # 병합
no_job.reset_index(drop=True, inplace=True) # 행으로 합쳤기 때문에 인덱스 초기화
no_job["house"] = (no_job["house1"] + no_job["house2"])/2 # 가사와 육아 병합
no_job["etc"] = (no_job["etc1"]+no_job["etc2"]+no_job["etc3"]+no_job["etc4"])/4 # 나머지 기타요인들을 기타무직으로 통합
no_job.drop(["house1", "house2", "etc1", "etc2", "etc3", "etc4"], axis=1, inplace=True) # 불필요한 열들 제거

### 모의 데이터 구축

In [ ]:
# 경제활동인구와 비경제활동인구 가르기
j = 50000*0.64  # 경제활동인구
nj = 50000*0.36  # 비경제활동인구

In [ ]:
# 신문 및 리포트에 따른 2022년 kb국민은행의 연령대별 이용률 대로 나누기
# 경제활동인구와 비경제활동인구에 각각 기존에 조사된 연령대별 비율적용
age_rate = [0.01, 0.26, 0.29, 0.24, 0.20]
j_age_rate = [j*age for age in age_rate] # 경제활동인구 10대 ~ 50대이상, 비경제활동인구 10대~50대이상
nj_age_rate = [nj*age for age in age_rate]

In [20]:
# 경제활동인구 먼저
# 지역별/직군별/성별로 구분된 데이터 비율활용
# 전 직군의 인원을 기준으로 각 직군의 비율이 결정되었다.
# 가장 첫 줄을 제외하고 1번 행부터 수도권, 강원도, 충청도, 경상도, 전라도, 제주도 순이며 0~5까지는 남성의 종사비율이고,
# 6~11까지는 여성의 종사비율이다.

# 연령대별 지역별 남성
jgar_m = [[j*jar for j in gj] for jar in j_age_rate for gj in gj_df.iloc[0:6, 1:].values.tolist()] # 10대 수도권 남성 1. , ... 50대 제주도 남성 10.
jgar_f = [[j*jar for j in gj] for jar in j_age_rate for gj in gj_df.iloc[6:12, 1:].values.tolist()] # 10대 수도권 여성 1. , ... 50대 제주도 여성 10.

In [19]:
# 비경제활동인구 - 위 처럼 한번에 묶인 것이 없기에 조금 복잡할 수 있다.
# 먼저 전국의 비경제활동인구의 지역별 활동상태 데이터를 가지고 온다.
njar = [[nj*jar for nj in rnj] for jar in nj_age_rate for rnj in rate_no_job.iloc[1:, 1:].values.tolist()] 
# 10대 수도권 (학생, 가사, 기타), 10대 강원도(학업, 가사, 기타), ..., 50대이상 제주도 (학업, 가사, 기타)

# 비경제활동의 경우 성별의 구분이 없어 추가해 주어야 한다.
# gender_df.iloc[2:, 2:]: 비경제활동인구의 남녀구분 비율 - 연령대별 남성비율이 먼저 나타난다.
'''
no_job 활용
'''
'''
1	0.360916	0.518239	0.021701	0.632224
2	0.639084	0.481761	0.978382	0.367776
'''
mnj = [0.52, 0.03, 0.63]
fnj = [0.48, 0.97, 0.37]

# 10대 수도권 학생 남성, 가사 남성, 기타 남성, 강원도 , ..., 50대이상 제주도 기타 남성
m_njar = [[mnj[i]*nja[i] for i in range(0, len(nja))] for nja in njar]
f_njar = [[fnj[i]*nja[i] for i in range(0, len(nja))] for nja in njar]

In [ ]:
# 완성된 가상 데이터 확인
sum(sum(jgar_m, [])+sum(jgar_f, [])+sum(m_njar, [])+sum(f_njar, []))

### 완성된 비율에 따라 인구수 분배

In [ ]:
# 각 연령대별 상황에 맞게 적용하기
ad_labels = ["수도권", "강원도", "충청도", "경상도", "전라도", "제주도"]
age_labels = ["10대", "20대", "30대", "40대", "50대이상"]

# 경제활동인구
job_labels = ["관리자", "전문가_및_관련종사자", "사무_종사자", "서비스_종사자", "판매_종사자",
              "농림어업_숙련_종사자", "기능원_및_관련기능_종사자", "장치_기계조작_및_조립_종사자", "단순_노무_종사자", "기타직업"]
# 비경제활동인구
no_labels = ["학생", "주부", "기타무직"]

def make_rate(jnj_list, sex, jnj):
    gender, age, address, occupation = [], [], [], []
    age_rep = len(sum(jnj_list, []))//len(age_labels) # 들어오는 리스트들의 크기에 맞게 연령대별 갯수 구분
    sub_age = ["10대"]*age_rep+["20대"]*age_rep+["30대"]*age_rep+["40대"]*age_rep+["50대이상"]*age_rep # 반복을 통한 길이 맞춤
    ad_rep = len(sum(jnj_list, []))//len(jnj_list) # 들어오는 리스트들의 크기에 맞게 행정구역별 반복횟수
    sub_ad = (["수도권"]*ad_rep+["강원도"]*ad_rep+["충청도"]*ad_rep+["경상도"]*ad_rep+["전라도"]*ad_rep+["제주도"]*ad_rep)*(len(age_labels)) # 행정구역의 크기맞추기
    job_rep = len(sum(jnj_list, []))//len(jnj) # 들어오는 리스트드르이 크기에 맞게 직업군별 반복횟수
    sub_job = jnj*job_rep # 직업군별 길이 맞추기
    
    for idx, i in enumerate(sum(jnj_list, [])): # 들어온 비율을 정수로 고친 후 전체 리스트에 추가
        i = int(round(i, 0))
        gender += [sex]*i
        age += [sub_age[idx]]*i
        occupation += [sub_job[idx]]*i
        address += [sub_ad[idx]]*i
    return gender, age, address, occupation


gender1, age1, address1, occupation1 = make_rate(jgar_m, "남성", job_labels) # 경제활동인구 남성
gender2, age2, address2, occupation2 = make_rate(jgar_f, "여성", job_labels) # 경제활동인구 여성
gender3, age3, address3, occupation3 = make_rate(m_njar, "남성", no_labels) # 비경제활동인구 남성
gender4, age4, address4, occupation4 = make_rate(f_njar, "여성", no_labels) # 비경제활동인구 여성

# 각 결과 합치기 - 모든 길이가 동일해야 한다.
gender = gender1+gender2+gender3+gender4
age = age1+age2+age3+age4
address = address1+address2+address3+address4
occupation = occupation1+occupation2+occupation3+occupation4

len(gender), len(age), len(address), len(occupation)

In [ ]:
# 데이터프레임 생성
user_df = pd.DataFrame({"gender": gender, "age": age, "occupation": occupation, "address": address})

### 추가조정

In [ ]:
# 10대는 학생으로 통일
user_df.loc[(user_df["occupation"]!="학생") & (user_df["age"]=="10대"), "occupation"] = "학생"

# 30대 이상의 학생은 기타로 처리
user_df.loc[(user_df["occupation"]=="학생") & (user_df["age"] == "30대"), "occupation"] = "기타무직"
user_df.loc[(user_df["occupation"]=="학생") & (user_df["age"] == "40대"), "occupation"] = "기타무직"
user_df.loc[(user_df["occupation"]=="학생") & (user_df["age"] == "50대이상"), "occupation"] = "기타무직"

### 기사 카테고리 지정

In [28]:
# 신문기사 경제부분 카테고리
labels = ["금융", "증시", "부동산", "국제경제", "소비자", "경제/정책"]

# 비슷한 카테고리별 묶기 + [소비자, 경제/정책]은 상대적으로 높은 정답 가중치 지정
labels_zip = [["금융", "증시"], ["부동산"], ["국제경제"], ["소비자", "경제/정책"]]

In [29]:
def make_mean():  # 평균 생성
    np.random.seed(random.randint(0, 100))  # 고정값 랜덤지정
    while True:
        random_number = np.random.normal(random.uniform(0.2, 0.6), random.uniform(0.01, 0.05))
        if 0.2 <= random_number <= 0.7:
            return random_number


def make_std(): # 표준편차 생성
    np.random.seed(random.randint(0, 100))
    while True:
        random_number = np.random.normal(random.uniform(0.01, 0.1), random.uniform(0.005, 0.01))
        if 0.01 <= random_number <= 0.1:
            return random_number

In [30]:
# ans_rate
def ans_rate(num_per_label, labels):
    weight, tot, ans = list(), list(), list()
    if "소비자" in labels:
        mean, std_dev = random.uniform(0.5, 0.8), random.uniform(0.01, 0.05)  
        # 소비자와 경제/정책의 경우 대부분의 국민들이 평균 이상의 지식을 가졌을 것이라고 판단
        
    else:
        # 다른 분류의 경우 관심도에 따라 깊이가 달라지기에 정규분포를 이루는 평균과 분산값을 구한다.
        mean, std_dev = make_mean(), make_std() #

    for label in labels:  # 비슷한 카테고리별 묶음에서 하나하나의 카테고리로 분리

        total = num_per_label.count(label)  # 한 사람이 푼 하나의 카테고리의 갯수
        tot.append(total)

        if total == 0:
            weight.append(0)
            ans.append(0)
        else:
            random_number = np.random.normal(mean, std_dev)  # 역시 정규분포를 이루는 평균과 분산
            rn = np.clip(random_number, 0, 1)  # 정답률은 최소 0에서 1까지
            answer = round(rn*total, 0)  # 정답 갯수 구하기

            ans.append(answer)
            weight.append(answer / total)  # 실제 정답률 기록

    return weight, tot, ans

# 라벨별 비중 랜덤으로 지정 - 유사라벨 묶음별 비중 랜덤 지정


def make_rate(labels_zip):  # 4개의 묶음
    rate_list = list()
    for labels in labels_zip:
        if "소비자" in labels:  # 위와 마찬가지로 생활경제나 경제일반은 정답률이 대부분 높기에 많은 비중으로 문제가 출제될 것이라 가정하였다.
            sub_rate = 0.3

        else:
            sub_rate = random.uniform(0, 0.7)  # 나머지 카테고리의 비율

        for label in labels:
            rate_list.append(sub_rate)        # 유사 카테고리끼리 동일한 비중 지정

    # rate 비율 조정
    rate_sum = sum(rate_list)  # 합이 1이 되도록 카테고리별 비율 지정
    rate = [rate/rate_sum for rate in rate_list]

    return rate

# 라벨 하나당 문제 출제 횟수와 정답 맞춘 횟수 랜덤으로 지정


def make_weight(num_per_label, labels, labels_zip):  # 한 사람당 카테고리별 정답비율, 정답 횟수, 전체 문제 횟수 기록
    weight, tot, ans = list(), list(), list()
    if len(num_per_label) == 0: # 한 사람에게 어떠한 카테고리의 문제도 출제되지 않은 경우
        weight = [0]*len(labels)
        tot = [0]*len(labels)
        ans = [0]*len(labels)
    else:
        for labels in labels_zip:
            sub_weight, sub_tot, sub_ans = ans_rate(num_per_label, labels)  # 라벨별 정답 갯수 무작위 선정
            weight += sub_weight
            tot += sub_tot
            ans += sub_ans

    return weight, tot, ans  # label별 정답률, label별 전체문제횟수, label별 정답문제횟수

def make_click_prob(rec_labels, labels, labels_zip): # 기사를 클릭한 횟수 / 라벨별로 기사가 추천된 횟수
    weight, click_news, rec_news = [], [], []
    if rec_labels == 0: # 처음 접하는 사용자의 경우
        weight = [0]*len(labels)
        click_news = [0]*len(labels)
        rec_news = [0]*len(labels)
        
    else:
        for labels in labels_zip:
            if "소비자" in labels:
                mean, std_dev = random.uniform(0.3, 0.7), random.uniform(0.01, 0.05)  
                # 소비자와 경제/정책의 경우 대부분의 국민들이 관심이 있어 뉴스를 눌러볼 확률이 높다고 판단
            else:
                # 다른 분류의 경우 관심도에 따라 깊이가 달라지기에 정규분포를 이루는 평균과 표준편차값을 구한다.
                mean, std_dev = make_mean(), make_std() # 

            for label in labels:  # 비슷한 카테고리별 묶음에서 하나하나의 카테고리로 분리

                total = rec_labels.count(label)  # 한 사람의 한 카테고리의 추천된 기사 전체 횟수
                rec_news.append(total*3)

                if total == 0:
                    weight.append(0) # 해당 카테고리의 기사를 추천한 적이 없으므로
                    click_news.append(0) # 해당 카테고리의 기사를 클릭할 수 없었으므로
                else:
                    random_number = np.random.normal(mean, std_dev)  # 역시 정규분포를 이루는 평균과 분산
                    rn = np.clip(random_number, 0, 1)  # 클릭률은 최소 0에서 1까지
                    click = round(rn*total*3, 0)  # 클릭 횟수 구하기
                    click_news.append(click)
                    weight.append(click / (total*3))  # 실제 클릭률 기록
            
    return weight, click_news, rec_news

In [31]:
# 행 하나 당, 즉 한 명당 비율 지정
np.random.seed(random.randint(0, 100))
def rate_label(num_rows, labels, labels_zip):
    df = pd.DataFrame(columns=labels)
    for i in range(0, num_rows):
        num = np.random.randint(1, 101) - 1  # 각 인원별 문제 출제 횟수
        labels_rate = make_rate(labels_zip)  # 인원당 문제 출제 비중
        labels_news = make_rate(labels_zip) # 인원당 라벨별 뉴스 추천 횟수
        num_per_label = np.random.choice(labels, p=labels_rate, size=num).tolist()  # rate 구성비율
        rec_labels = np.random.choice(labels, p=labels_news, size=num).tolist() # 추천된 기사의 카테고리

        weight, tot, ans = make_weight(num_per_label, labels, labels_zip)  # 라벨이 무작위로 들어간 리스트, label
        click_weight, click_news, rec_news = make_click_prob(rec_labels, labels, labels_zip) # 클릭률, 클릭한 횟수, 추천한 횟수

        tot_labels = [f"{label}_tot" for label in labels] # 카테고리별 총 출제 갯수
        ans_labels = [f"{label}_ans" for label in labels] # 정답 횟수
        acc_avg = sum(weight) / 6 # 평균 정답 수
        click_labels = [f"{label}_click" for label in labels] # 클릭한 횟수 
        label_rec = [f"rec_{label}_news" for label in labels] # 추천받은 횟수
        click_rate = [f"{label}_click_probs" for label in labels] # 클릭률

        df_sub = pd.DataFrame([weight+[acc_avg]+click_weight+tot+ans+[num]+click_news+rec_news],
                              columns=labels+["acc_avg"]+click_rate+tot_labels+ans_labels+["total"]+click_labels+label_rec)
        df = pd.concat([df, df_sub], axis=0)
    df = df.reset_index(drop=True)
    return df

In [ ]:
# 데이터프레임 섞기 (shuffle)
shuffled_user_df = user_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# 비율 생성
weights = rate_label(shuffled_user_df.shape[0], labels, labels_zip)

# user_df와 병합
user_data = pd.concat([shuffled_user_df, weights], axis=1)
user_data["user_id"] = [f"user_{i}" for i in range(1, user_data.shape[0]+1)]
user_data.set_index("user_id", inplace=True)
user_data.reset_index(drop=False, inplace=True)

In [ ]:
# rating을 위해 필요한 부분만 분리
split_user_data = user_data.iloc[:, 0:18]

### 저장

In [ ]:
# 파일 저장
user_data.to_csv(os.path.join(save_path, "user_db.csv"), encoding="utf-8", index=False)
split_user_data.to_csv(os.path.join(save_path, "split_db.csv"), encoding="utf-8", index=False)